In [479]:
"""
task: calculate the KPI of models automatically
author: Zheng Wei
latest modification date: 2018/06/08
"""
import numpy as np
import pandas as pd
import os
file_dir = ("C:/Users/zhewei/Desktop/KPI/")
file_list = [files for root, dirs, files in os.walk(file_dir)][0]
head = ["Model", "Model name", "Segment name", "Update date", "Delay month", "Selected month", "OP", "Catch rate", "Target",
        "target with 5% tolerance(absolutely)", "KPI with no tolerance", "KPI with 5% tolerance(absolutely)"]
report = dict(zip(head, [list() for i in range(len(head))]))
for model_file in file_list:
    calc_kpi(file_dir + model_file)
report = pd.DataFrame(report, columns=["Model", "Model name", "Segment name", "Update date", "Delay month", "Selected month", "OP", "Catch rate", "Target",
        "target with 5% tolerance(absolutely)", "KPI with no tolerance", "KPI with 5% tolerance(absolutely)"])
output_monthly(np.array(report), head)

['2017/11/01', '2017/12/01', '2018/01/01', '2018/02/01', '2018/03/01', '2018/04/01', '2018/05/01']
['2017/12/01', '2018/01/01', '2018/02/01', '2018/03/01', '2018/04/01', '2018/05/01']
['2017/11/01', '2017/12/01', '2018/01/01', '2018/02/01', '2018/03/01', '2018/04/01', '2018/05/01']


In [469]:
from xlwt import *

def output_monthly(report, head, date = "201801"):
    """
    Output summarized excel monthly.
    
    @param report: a two-dims array generated from pandas.dataframe: report.
    @param head: a list generated from cloumns of report, a object of pandas dataframe.
    """
    row, column = report.shape
    model_list_reverse = list(report[:,0])[::-1]
    catch_rate_index = head.index("Catch rate")
    target_inedx = head.index("target with 5% tolerance(absolutely)")
    KPI_index = head.index("KPI with 5% tolerance(absolutely)")
    
    # set style 
    alignment = Alignment() # Create Alignment
    alignment.horz = xlwt.Alignment.HORZ_CENTER # May be: HORZ_GENERAL, HORZ_LEFT, HORZ_CENTER, HORZ_RIGHT, HORZ_FILLED, HORZ_JUSTIFIED, HORZ_CENTER_ACROSS_SEL, HORZ_DISTRIBUTED
    alignment.vert = xlwt.Alignment.VERT_CENTER # May be: VERT_TOP, VERT_CENTER, VERT_BOTTOM, VERT_JUSTIFIED, VERT_DISTRIBUTED
    font = Font() # 为样式创建字体
    font.name = 'Times New Roman' 
    font.colour_index = 2
    font.underline = True  # 下划线
    pattern = Pattern()
    pattern.pattern = Pattern.SOLID_PATTERN  # 设置其模式为实型    
    pattern.pattern_fore_colour = 5
    
    # different styles
    # normal style
    style = xlwt.XFStyle() # Create Style
    style.alignment = alignment
    # head style
    font1 = Font() 
    font1.bold = True
    font1.height = 200
    style_head = xlwt.XFStyle()
    style_head.font = font1
    style_head.alignment = alignment
    
    # Style1 
    style1 = XFStyle() # 初始化样式
    style1.font = font
    style1.alignment = alignment
    # Style2
    style2 = XFStyle()
    style2.alignment = alignment
    style2.pattern = pattern
    
    workbook = xlwt.Workbook(encoding='utf-8')
    worksheet = workbook.add_sheet("sheet1")
    width = 256 * 18
    tall_style = xlwt.easyxf('font:height 360;') # 36pt,类型小初的字号
    
    for i in range(row + 1):
        worksheet.row(i).set_style(tall_style)
        merge_num = 1
        for j in range(column):
            if i == 0:
                temp_col = worksheet.col(j)
                temp_col.width = width   
                worksheet.write(i, j, head[j], style_head)
                continue
            if j == 0:
                merge_num = row + 1 - (model_list_reverse.index(report[i-1,j]) + i)
                try:
                    worksheet.write_merge(i, i + merge_num - 1, j, j, report[i-1][j], style)
                except:
                    continue
            elif j == catch_rate_index:
                if report[i-1][j] < report[i-1][target_inedx]:
                    worksheet.write(i, j, report[i-1,j], style1)
                else:
                    worksheet.write(i, j, report[i-1,j], style)
            elif j == KPI_index:
                if int(report[i-1][j]):
                    worksheet.write(i, j, report[i-1,j], style)
                else:
                    worksheet.write(i, j, report[i-1,j], style2)
            else:
                try:
                    if report[i-1][j] == report[i - 1 + merge_num - 1][j]:
                        worksheet.write_merge(i, i + merge_num - 1, j, j, report[i-1][j], style)
                    else:
                        worksheet.write(i, j, report[i-1][j], style)
                except:
                    continue

    workbook.save("/Users/zhewei/Desktop/KPI_result/KPI_monthly" + date + ".xls")

In [468]:
def calc_kpi(model_file):
    """
    Given a file path of certain model, calculate its KPI. 
    """
    global report  # use global variable report to update kpi.
    data = pd.read_excel(model_file)
    model_name = list(data["score"])[0]
    if "Tier1" in model_name:
        grap_tier(model_name, data)
        return
    # Judge whether the number of segments is one
    more_segs = len(list(data["dim_value"])[0].split(',')) - 1
    if more_segs:
        temp_list = sorted([item.split(',') for item in list(data["dim_value"])], key=lambda x: x[1])
        seg_names = list(set([item.split(',')[1] for item in list(data["dim_value"])]))
        delay_month = 2
        for seg_name in seg_names:
            report["Model"].append(model_name[:model_name.index('_')])
            report["Model name"].append(model_name.replace('_SCORE', '').replace('_score', ''))
            report["Delay month"].append(delay_month)
            report["Segment name"].append(seg_name)
            grap_segs(model_name, seg_name, data, delay_month)
    else:
        report["Model"].append(model_name[:model_name.index('_')])
        report["Model name"].append(model_name.replace('_SCORE', '').replace('_score', ''))
        delay_month = 2
        if "ATOM" in model_name:
            delay_month = 1
            report["Segment name"].append("Atom Seg2")
        elif "CAM" in model_name:
            report["Segment name"].append("Cam eligible")
        elif "ARM" in model_name:
            report["Segment name"].append("iach & celv")
        report["Delay month"].append(delay_month)
        grap_singal_seg(model_name, data, delay_month)

In [467]:
def grap_tier(model_name, data):
    """
    For Tier1 models, add corresponded value into report. 
    
    @param model_name: a string of model name.
    @param data: Dataframe generated from excel. 
    """
    global report
    seg_list = list(data["file_name"])
    seg_set = [item.split('_')[1] for item in list(set(seg_list))]
    time_list = [time.split('/')[:-1] for time in list(data["table"])]
    rank_value = list(data["ptotbad"])
    delay_month = 2
    update_date = list(data["update_date"])[0]
    year, month =[int(item) for item in update_date.split('-')[:-1]]
    month -= 4
    update_date = "2018-01-31"
    for seg_name in seg_set:
        report["Model"].append(model_name[:model_name.index('_')])
        report["Model name"].append(model_name.replace('_SCORE', '').replace('_score', ''))
        report["Segment name"].append(seg_name)
        report["Delay month"].append(delay_month)
        report["Update date"].append(update_date)
        corresponded_index = [i for i in range(len(seg_list)) if seg_name in seg_list[i]]
        key = [tuple(time_list[i]) for i in corresponded_index]
        value = [rank_value[i] for i in corresponded_index]
        op_dict = dict(zip(key, value))
        true_op = 0
        if month == 12:
            true_op = op_dict[str(year), '10']
            report["Selected month"].append('10')
        elif month == 1:
            true_op = op_dict[str(year-1), '11']
            report["Selected month"].append('11')
        elif month == 2: 
            true_op = op_dict[str(year-1), '12']
            report["Selected month"].append('12')
        else:
            true_op = op_dict[str(year), '0' + str(month-2)]
            report["Selected month"].append(str(month-2))
        report["Catch rate"].append("%.1f%%" % (true_op * 100))
        report["OP"].append("# OP {num}% Dol Catch Rate".format(num="25"))
        target_op = 0.95
        report["Target"].append("%.0f%%" % (target_op * 100))
        report["target with 5% tolerance(absolutely)"].append("%.0f%%" % ((target_op-0.05) * 100))
        report["KPI with no tolerance"].append(int(true_op > target_op))
        report["KPI with 5% tolerance(absolutely)"].append(int(true_op > (target_op - 0.05)))

In [475]:
def grap_singal_seg(model_name, data, delay_month):
    """
    For models with signal segment, add corresponded value into report. 
    
    @param model_name: a string of model name.
    @param data: Dataframe generated from excel. 
    @param delay_month: manually setted num.
    """
    global report
    update_date = list(data["update_date"])[0]
    year, month = [int(item) for item in update_date.split('-')][:-1]
    month -= 4
    update_date = "2018-01-31"
    report["Update date"].append(update_date)
    key = [tuple(item.split('/')[:-1]) for item in list(data["dim_value"])]
    op_name = ""
    for item in data.columns:
        if "OP" in item:
            op_name = item
            value = list(data[item])
            break
    op_dict = dict(zip(key, value))
    true_op = 0
    if delay_month == 1:
        if month == 10:
            true_op = op_dict[str(year), '09']
            report["Selected month"].append('09')
        elif month == 1:
            true_op = op_dict[str(year-1), '12']
            report["Selected month"].append('12')
        else:
            true_op = op_dict[str(year), '0' + str(month-1)]
            report["Selected month"].append(str(month-1))
    elif delay_month == 2:
        if month == 12:
            true_op = op_dict[str(year), '10']
            report["Selected month"].append('10')
        elif month == 1:
            true_op = op_dict[str(year-1), '11']
            report["Selected month"].append('11')
        elif month == 2: 
            true_op = op_dict[str(year-1), '12']
            report["Selected month"].append('12')
        else:
            true_op = op_dict[str(year), '0' + str(month-2)]
            report["Selected month"].append(str(month-2))
    else:
        pass
    report["Catch rate"].append("%.1f%%" % (true_op * 100))
    target_op = 0
    if "ATOM" in model_name:
        target_op = 0.80
        report["OP"].append("$ OP 4.5% Dol Catch Rate")
    elif "ARM" in model_name:
        target_op = 0.85
        report["OP"].append("$ OP 4.5% nLoss Catch Rate")
    elif "CAM" in model_name:
        target_op = 0.85
        report["OP"].append("$ OP 6% Dol Catch Rate")
    else:
        target_op = 0
        report["OP"].append("It's a new model, Please check code")
    report["Target"].append("%.0f%%" % (target_op * 100))
    report["target with 5% tolerance(absolutely)"].append("%.0f%%" % ((target_op-0.05) * 100))
    report["KPI with no tolerance"].append(int(true_op > target_op))
    report["KPI with 5% tolerance(absolutely)"].append(int(true_op > (target_op - 0.05)))
    
def grap_segs(model_name, seg_name, data, delay_month):
    """
    For models with more than one segment, add corresponded value into report. 
    
    @param model_name: a string of model name.
    @param seg_name: a string of segment name.
    @param data: Dataframe generated from excel. 
    @param delay_month: manually setted num.
    """
    global report
    update_date = list(data["update_date"])[0]
    year, month = [int(item) for item in update_date.split('-')][:-1]
    month -= 4
    update_date = "2018-01-31"
    report["Update date"].append(update_date)
    dim_value = list(data["dim_value"])
    corrsponded_index = [i for i in range(len(dim_value)) if seg_name in dim_value[i]]
    key = [tuple(dim_value[i].split(',')[0].split('-')) for i in corrsponded_index]
    op_name = ""
    if "YS" in seg_name:
        op_name = "OP4"
    elif "MM" in model_name:
        op_name = "OP1"
    elif "EXAM" in model_name:
        op_name = "OP2"
    else:
        pass
    op_value = list(data[op_name])
    value = [op_value[i] for i in corrsponded_index] 
    op_dict = dict(zip(key, value))
    true_op = 0
    if delay_month == 1:
        if month == 10:
            true_op = op_dict[str(year), '9']
            report["Selected month"].append('9')
        elif month == 1:
            true_op = op_dict[str(year-1), '12']
            report["Selected month"].append('12')
        else:
            true_op = op_dict[str(year), str(month-1)]
            report["Selected month"].append(str(month-1))
    elif delay_month == 2:
        if month == 12:
            true_op = op_dict[str(year), '10']
            report["Selected month"].append('10')
        elif month == 1:
            true_op = op_dict[str(year-1), '11']
            report["Selected month"].append('11')
        elif month == 2: 
            true_op = op_dict[str(year-1), '12']
            report["Selected month"].append('12')
        else:
            true_op = op_dict[str(year), str(month-2)]
            report["Selected month"].append(str(month-2))
    else:
        pass
    report["Catch rate"].append("%.1f%%" % (true_op * 100))
    target_op = 0
    if "MM" in model_name:
        report["OP"].append("# OP {num}% gloss Catch Rate".format(num=op_name[2:]))
        if "SME" in seg_name:
            target_op = 0.26
        elif "CS" in seg_name:
            target_op = 0.302
        elif "YS" in seg_name:
            target_op = 0.56
        else:
            pass
    elif "EXAM" in model_name:
        report["OP"].append("$ OP {num}% Dol Catch Rate".format(num=op_name[2:]))
        if "SME" in seg_name:
            target_op = 0.4
        elif "CS" in seg_name:
            target_op = 0.54
        elif "YS" in seg_name:
            target_op = 0.44
        else:
            pass
    report["Target"].append("%.0f%%" % (target_op * 100))
    report["target with 5% tolerance(absolutely)"].append("%.0f%%" % ((target_op-0.05) * 100))
    report["KPI with no tolerance"].append(int(true_op > target_op))
    report["KPI with 5% tolerance(absolutely)"].append(int(true_op > (target_op - 0.05)))